In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import TensorDataset, DataLoader


# Load the data and create a DataFrame
D = np.load('../data/Lorenz_datay')
df = pd.DataFrame(D)
#df.columns = ["time", "x", "velocity"]  
df

FileNotFoundError: [Errno 2] No such file or directory: '../data/Lorenz_datay'

In [ ]:

# Extract position and velocity as separate time series
timeseries_p = df[["position"]].values.astype('float32') 
timeseries_v = df[["velocity"]].values.astype('float32') 

# Extract time series for overall data
times = df[["time"]].values.astype('float32')
timeseries =  df[["position", "velocity"]]

# train-test split for time series
train_size = int(len(timeseries_p) * 0.85)
test_size = len(timeseries_p) - train_size
p_train, p_test = timeseries_p[:train_size], timeseries_p[train_size:]
v_train, v_test = timeseries_v[:train_size], timeseries_v[train_size:]
t_train, t_test = times[:train_size],times[train_size:]


In [ ]:

# Function to create the dataset
def create_dataset(dataset_p, dataset_v,  lookback):
    X, y = [], []
    for i in range(len(dataset_p)-lookback):
        # Create feature by stacking lookback points of position and velocity
        feature = np.column_stack((dataset_p[i:i+lookback], dataset_v[i:i+lookback]))
        # Create target by stacking lookback+1 points of position and velocity
        target = np.column_stack((dataset_p[i+1:i+lookback+1],dataset_v[i+1:i+lookback+1]))  
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)
 
lookback = 4
X_train, y_train  = create_dataset(p_train, v_train,  lookback=lookback)